In [4]:
!pip install openpyxl
!pip install matplotlib
!pip install seaborn

import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision.models as models
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
# Import data
brain_slices = np.load('/rds/projects/b/bowmanh-msc-project-group-2023to2024/Yifei_/revised_stitched_ws_images_v5.npy',allow_pickle=True)
df = pd.read_excel('/rds/projects/b/bowmanh-msc-project-group-2023to2024/Yifei_/Patients_CathyExport_processed_v5.xlsx', sheet_name='master_run')

In [6]:
patient_data_nn = df[:1072]

In [7]:
scanners_to_exclude = ['SON', 'NCL', 'EXT', 'UCH']
training_list = [1,2,3,5]
test_list = [4]
s_train_val = brain_slices[patient_data_nn.NewGroups.isin(training_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]
s_test = brain_slices[patient_data_nn.NewGroups.isin(test_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]

print("X_train_val:", s_train_val.shape)
print("X_test:", s_test.shape)

X_train_val: (600, 632, 760)
X_test: (158, 632, 760)


In [8]:
labels_train = patient_data_nn[patient_data_nn.NewGroups.isin(training_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]['SpkPicDescTOTTS']
labels_test = patient_data_nn[patient_data_nn.NewGroups.isin(test_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]['SpkPicDescTOTTS']

y_train_val = (labels_train>=60).astype(int)
y_test = (labels_test>=60).astype(int)

print("y_train_val:", len(y_train_val))
print("y_test:", len(y_test))

y_train_val: 600
y_test: 158


In [9]:
# Preprocess the image data
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
#     transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_slices(slices):
    tensors = [preprocess(torch.tensor(slice, dtype=torch.float).unsqueeze(0)) for slice in slices]
    return torch.stack(tensors)

X_train_test = preprocess_slices(s_train_val).to(device)
y_train_test = torch.from_numpy(y_train_val.values).long().to(device)
X_test = preprocess_slices(s_test).to(device)
y_test = torch.from_numpy(y_test.values).long().to(device)

print(f"Number of Training/Validation: {len(y_train_test)}")
print(f"Number of Testing: {len(y_test)}")

# Preprocess the image data (assuming preprocess_slices function is defined)
print(f"Train/Validation Images Shape: {X_train_test.shape}")
print(f"Test Images Shape: {X_test.shape}")

test_data = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

Number of Training/Validation: 600
Number of Testing: 158
Train/Validation Images Shape: torch.Size([600, 3, 256, 307])
Test Images Shape: torch.Size([158, 3, 256, 307])


In [10]:
# Define the model
class_names = ['below','above']

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(256, len(class_names))
)

model = model.to(device)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

/rds/homes/y/yxm326/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/rds/homes/y/yxm326/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Earlystopping strateges:
#### 1. UP for α length
#### 2. Generalization Loss over α
#### 3. improvement of 2
#### 4. cross
#### 5. run for 200 epochs for minimum

In [11]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [12]:
class EarlyStoppingAvgLoss:
    def __init__(self, patience=5, avg_len=10):
        self.patience = patience
        self.avg_len = avg_len
        self.best_avg_loss = float('inf')
        self.losses = []
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        self.losses.append(val_loss)
        if len(self.losses) > self.avg_len:
            self.losses = self.losses[-self.avg_len:]
        avg_loss = sum(self.losses) / len(self.losses)
        if avg_loss < self.best_avg_loss:
            self.best_avg_loss = avg_loss
            self.counter = 0
        else:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")

            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [13]:
class EarlyStoppingWeight:
    def __init__(self, patience=5, alpha=0.5):
        self.patience = patience
        self.alpha = alpha
        self.best_avg_loss = float('inf')
        self.avg_loss = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.avg_loss == 0:
            self.avg_loss = val_loss
        else:
            self.last_loss = self.avg_loss
            self.avg_loss = self.alpha * val_loss + (1 - self.alpha) * self.avg_loss
        if self.avg_loss < self.best_avg_loss:
            self.best_avg_loss = self.avg_loss
        elif val_loss < self.last_loss:
            self.counter = 0
        else:
            self.counter += 1
#             print(f"INFO: Early stopping counter {self.counter} of {self.patience}")

            if self.counter >= self.patience:
#                 print('INFO: Early stopping')
                self.early_stop = True

### GL

In [14]:
class EarlyStoppingGL:
    def __init__(self, alpha=0.5):
        """
        Initialize the early stopping criteria based on Generalization Loss (GL).

        Parameters:
        alpha (float): Threshold for the GL to trigger early stopping.
        """
        self.alpha = alpha
        self.best_loss = float('inf')
        self.early_stop = False
        self.stop_epoch = None
        self.val_losses = []

    def GL(self, val_loss, min_val_loss):
        return 100 * (val_loss / min_val_loss - 1)

    def __call__(self, val_loss):
        self.val_losses.append(val_loss)
        if val_loss < self.best_loss:
            self.best_loss = val_loss

        GL_value = self.GL(val_loss, self.best_loss)
        if GL_value > self.alpha:
            self.early_stop = True
            self.stop_epoch = len(self.val_losses)

In [15]:
class EarlyStoppingGLPQRatio:
    def __init__(self, alpha=0.95, k=5):
        """
        Initialize the early stopping criteria based on GL/PQ ratio.

        Parameters:
        alpha (float): Threshold for the GL/PQ ratio to trigger early stopping.
        k (int): Number of epochs over which to calculate the PQ value.
        """
        self.alpha = alpha
        self.k = k
        self.best_loss = float('inf')
        self.val_losses = []
        self.train_losses = []
        self.early_stop = False
        self.stop_epoch = None

    def GL(self, val_loss, min_val_loss):
        return 1000 * (val_loss / min_val_loss - 1)

    def PQ(self, train_losses):
        min_train_loss = min(train_losses)
        avg_train_loss = sum(train_losses) / len(train_losses)
        return 100 * (avg_train_loss / min_train_loss - 1)

    def __call__(self, val_loss, train_loss):
        self.val_losses.append(val_loss)
        self.train_losses.append(train_loss)

        if val_loss < self.best_loss:
            self.best_loss = val_loss

        else:
            GL_value = self.GL(val_loss, self.best_loss)
            PQ_value = self.PQ(self.train_losses[-self.k:])
            if GL_value / PQ_value > self.alpha:
                self.early_stop = True
                self.stop_epoch = len(self.val_losses)

In [16]:
class EarlyStoppingCross:
    def __init__(self):
        """
        Initialize the early stopping criteria based on the last crossing point
        of the train loss and validation loss curves after training for 100 epochs.
        """
        self.train_losses = []
        self.val_losses = []
        self.best_epoch = None

    def __call__(self, train_loss, val_loss):
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)

        if len(self.train_losses) >= 100:
            for epoch in range(99, -1, -1):
                if self.train_losses[epoch] > self.val_losses[epoch]:
                    self.best_epoch = epoch + 1
                    break

        if self.best_opoch == None:
            self.best_epoch = self.val_losses.index(min(self.val_losses)) + 1

    def get_best_epoch(self):
        return self.best_epoch

In [17]:
class EarlyStoppingValLoss:
    def __init__(self, s=20):
        """
        Initialize the early stopping criteria based on consecutive increases in validation loss.

        Parameters:
        s (int): Number of consecutive epochs with increasing validation loss to trigger early stopping.
        """
        self.s = s
        self.best_loss = float('inf')
        self.val_losses = []
        self.early_stop = False
        self.stop_epoch = None

    def __call__(self, val_loss):
        """
        Check if training should be stopped based on consecutive increases in validation loss.

        Parameters:
        val_loss (float): Current validation loss.
        """
        self.val_losses.append(val_loss)
        if val_loss < self.best_loss:
            self.best_loss = val_loss

        if len(self.val_losses) > self.s:
            increasing = all(x < y for x, y in zip(self.val_losses[-self.s:], self.val_losses[-self.s + 1:]))
            if increasing:
                self.early_stop = True
                self.stop_epoch = len(self.val_losses)

In [18]:
best_val_loss = float('inf')   # Initialize the best validation loss to positive infinity
model_path = 'best_model.pth'  # Path to save the best model weights
n_epochs = 50
# early_stopping = EarlyStopping(patience=20, min_delta=0.01)
# early_stopping = EarlyStoppingAvgLoss(patience=20, avg_len=3)
# early_stopping = EarlyStoppingEWMA(patience=10, alpha=0.5)

# early_stopping_gl = EarlyStoppingGL()
# early_stopping_glqr = EarlyStoppingGLPQRatio()
# early_stopping_val_loss = EarlyStoppingValLoss()

In [ ]:
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.model_selection import KFold

# Assume necessary imports and definitions (device, class_names, etc.)

def create_model():
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, len(class_names))
    )
    return model.to(device)

def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        predicted = (output > 0).float()
        total_train += target.size(0) * target.size(1)
        correct_train += (predicted == target).float().sum()
    
    return train_loss / len(train_loader.dataset), 100 * correct_train / total_train

def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
            
            output = model(data)
            loss = criterion(output, target)
            
            val_loss += loss.item() * data.size(0)
            predicted = (output > 0).float()
            total_val += target.size(0) * target.size(1)
            correct_val += (predicted == target).float().sum()
    
    return val_loss / len(val_loader.dataset), 100 * correct_val / total_val

def train_and_validate(X_train, y_train, X_val, y_val, n_epochs):
    model = create_model()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_val, y_val), batch_size=32)

    best_val_loss = float('inf')
    best_model_state = None
    train_losses, val_losses = [], []
    val_accuracies = []
    best_epoch = None

    for epoch in range(n_epochs):
        train_loss, train_accuracy = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_accuracy = validate(model, val_loader, criterion)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        
        print(f'  Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, Val Accuracy: {val_accuracy:.2f}%')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            
    if len(train_losses) >= 100:
        for epoch in range(99, -1, -1):
            if train_losses[epoch] > val_losses[epoch]:
                best_epoch = epoch
                break

    if best_epoch == None:
        best_epoch = val_losses.index(min(val_losses))

    return best_model_state, val_losses[best_epoch], val_accuracies[best_epoch]

def test_model(model, X_test, y_test):
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=32)
    criterion = nn.BCEWithLogitsLoss()
    test_loss, test_accuracy = validate(model, test_loader, criterion)
    return test_loss, test_accuracy

def run_experiment(X, y, X_test, y_test, num_runs, k_folds, n_epochs):
    all_results = []

    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs}")
        kfold = KFold(n_splits=k_folds, shuffle=True, random_state=run)
        
        run_fold_losses = []
        run_fold_accuracies = []
        best_run_model = None
        best_run_loss = float('inf')
        
        for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
            print(f'Fold {fold + 1}')
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            # Train and validate model
            best_model_state, best_val_loss, val_accuracy = train_and_validate(X_train, y_train, X_val, y_val, n_epochs)
            run_fold_losses.append(best_val_loss)
            run_fold_accuracies.append(val_accuracy)
            print(f'Fold {fold + 1} - Best Validation Loss: {best_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')
            
            if best_val_loss < best_run_loss:
                best_run_loss = best_val_loss
                best_run_model = best_model_state
        print('\n')
        print(f"============== Run {run + 1} Info ============== ")
        avg_fold_loss = sum(run_fold_losses) / k_folds
        avg_fold_accuracy = sum(run_fold_accuracies) / k_folds
        print(f'Run {run + 1} - Average Validation Loss: {avg_fold_loss:.4f}, Average Validation Accuracy: {avg_fold_accuracy:.2f}%')
        
        # Test model on the test set
        best_model = create_model()
        best_model.load_state_dict(best_run_model)
        test_loss, test_accuracy = test_model(best_model, X_test, y_test)
        print(f"Run {run + 1} - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
        print(f"============== Run {run + 1} End  ============== ")
        print('\n')
        
        # Record the result
        all_results.append({
            'run': run + 1,
            'fold_losses': run_fold_losses,
            'fold_accuracies': run_fold_accuracies,
            'avg_val_loss': avg_fold_loss,
            'avg_val_accuracy': avg_fold_accuracy,
            'test_loss': test_loss,
            'test_accuracy': test_accuracy
        })
    
    return all_results
        
# Main execution
if __name__ == "__main__":
    # Define hyperparameters
    num_runs = 50
    k_folds = 4
    n_epochs = 100

    run_results = run_experiment(
        X_train_test, y_train_test, X_test, y_test, num_runs, k_folds, n_epochs
    )

    with open('Cross_result_detailed.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Run', 'Fold', 'Validation Loss', 'Validation Accuracy', 'Avg Validation Loss', 'Avg Validation Accuracy', 'Test Loss', 'Test Accuracy'])
        for result in run_results:
            for fold in range(k_folds):
                writer.writerow([
                    result['run'],
                    fold + 1,
                    result['fold_losses'][fold],
                    result['fold_accuracies'][fold],
                    result['avg_val_loss'],
                    result['avg_val_accuracy'],
                    result['test_loss'],
                    result['test_accuracy']
                ])

    with open('Cross_result_summary.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Run', 'Avg Validation Loss', 'Avg Validation Accuracy', 'Test Loss', 'Test Accuracy'])
        for result in run_results:
            writer.writerow([
                result['run'],
                result['avg_val_loss'],
                result['avg_val_accuracy'],
                result['test_loss'],
                result['test_accuracy']
            ])

    print("Detailed results saved to 'Cross_result_detailed.csv'")
    print("Summary results saved to 'Cross_result_summary.csv'")

Run 1/50
Fold 1
  Epoch 1: Train Loss: 0.6938, Val Loss: 0.6836, Train Accuracy: 53.67%, Val Accuracy: 54.33%
  Epoch 2: Train Loss: 0.6798, Val Loss: 0.6747, Train Accuracy: 57.11%, Val Accuracy: 56.33%
  Epoch 3: Train Loss: 0.6661, Val Loss: 0.6665, Train Accuracy: 59.56%, Val Accuracy: 59.00%
  Epoch 4: Train Loss: 0.6486, Val Loss: 0.6555, Train Accuracy: 65.67%, Val Accuracy: 62.33%
  Epoch 5: Train Loss: 0.6427, Val Loss: 0.6513, Train Accuracy: 66.78%, Val Accuracy: 63.67%
  Epoch 6: Train Loss: 0.6440, Val Loss: 0.6477, Train Accuracy: 66.67%, Val Accuracy: 64.67%
  Epoch 7: Train Loss: 0.6392, Val Loss: 0.6461, Train Accuracy: 66.33%, Val Accuracy: 64.67%
  Epoch 8: Train Loss: 0.6299, Val Loss: 0.6445, Train Accuracy: 66.89%, Val Accuracy: 65.00%
  Epoch 9: Train Loss: 0.6308, Val Loss: 0.6443, Train Accuracy: 67.00%, Val Accuracy: 65.00%
  Epoch 10: Train Loss: 0.6291, Val Loss: 0.6434, Train Accuracy: 67.11%, Val Accuracy: 65.00%
  Epoch 11: Train Loss: 0.6275, Val Loss: 0

# End